<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/bert_model_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip3 -q install datasets transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### I made a program which picks random texts from one of the datasets we used in this course and produces two files: one with segments of texts with one [MASK] and one with the correct answers and tried to guess the words without looking at the latter file and then compare your answers with the correct ones.  I used IMDB dataset and used two separate files called Masked_File & AnswersMasked to keep the one with segments of texts with one [MASK] and one with the correct answer. By comparing these two, I also found the accuracy for this at the end by seeing if it guessed it correctly in the top10.


In [ ]:
import pandas as pd
import transformers
import datasets
import torch
import random


In [ ]:
imdb_df1=pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")
imdb_df1

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:


imdb_df1["review"] = imdb_df1["review"].str.replace("<br /><br />", "")
reviews_all=imdb_df1["review"]
print(reviews_all.head(4))

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. The filming tec...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
Name: review, dtype: object


In [ ]:
no_lines=10
random_lines=reviews_all.sample(n=no_lines)
#random_lines=random.randint(0,20)
print(random_lines)

8877     Unfortunately many consumers who write reviews...
16773    Where to begin. This movie started out as some...
44611    The acting is good, the action is good, and so...
408      I watched this movie based on the good reviews...
12781    They really can't get stupider than this film ...
46725    Once upon a time Hollywood produced live-actio...
42958    I wasn't expecting much, and, to be honest, I ...
21568    I really liked this movie. I've read a few of ...
8518     I must admit, at first I wasn't expecting anyt...
14740    Jesse yet again delivers, after almost 12 mont...
Name: review, dtype: object


In [ ]:
reviews_masked=[]
masked_positions=[]
masked_words=[]
for i in range(len(random_lines)):
  str1=""
  random_position=random.randint(1,len(random_lines))
  words=random_lines.iloc[i].split(" ")
  #print(words)
  for j in range(len(words)):
    if random_position==j:
      str1+="[MASK] "
      masked_positions.append(j+1)
      masked_words.append(words[j])
    else:
      str1+=words[j]+" "
  reviews_masked.append(str1)




In [ ]:
Masked_file=[]
Answers=[]

Answers=pd.DataFrame({'masked_position': masked_positions, 'masked_words':masked_words})
Masked_file=pd.DataFrame({'masked_sentences': reviews_masked})

#print(reviews_masked)
#print(len(reviews_masked))
#print(masked_positions)
#print(masked_words)

Answers.to_csv("AnswersofMasked.csv",index=False, sep=' ')
Masked_file.to_csv("Masked_file.csv",index=False, sep=' ')



In [ ]:
reviews_ans=pd.read_csv("/content/AnswersofMasked.csv",delim_whitespace=True)
reviews_masked=pd.read_csv("/content/Masked_file.csv", delim_whitespace=True)
reviews_ans

,masked_position,masked_words
0,5,write
1,3,begin.
2,9,and
3,7,the
4,2,really
5,7,"live-action,"
6,5,"and,"
7,2,really
8,3,"admit,"
9,9,of


# Basic BERT operations


In [ ]:
tokenizer=transformers.AutoTokenizer.from_pretrained("bert-base-cased") #you can also use the trusty "TurkuNLP/bert-base-finnish-cased-v1"
#bert=transformers.AutoModel.from_pretrained("bert-base-cased") #"TurkuNLP/bert-base-finnish-cased-v1" if you run this in Finnish
bert=transformers.AutoModelForPreTraining.from_pretrained("bert-base-cased")


In [ ]:
# We will be running the model directly, so let's use return_tensors="pt" to get torch tensors rather than Python lists
#texts=["Dogs like to [MASK] cats. They taste good.","Bad joke!"]
correct=0
total=len(reviews_ans)
reviews_ans["masked_position"]=pd.to_numeric(reviews_ans["masked_position"])
reviews_ans["masked_words"]=reviews_ans["masked_words"].astype('str')
reviews_masked["masked_sentences"]=reviews_masked["masked_sentences"].astype('str')
print(reviews_ans.dtypes)
print(reviews_masked.dtypes)
for i in range(len(reviews_masked)-1):
  texts=str(reviews_masked.iloc[i,0])
  t=tokenizer(texts,padding=True, truncation=True, return_tensors="pt")

  bert_out=bert(
    input_ids=t["input_ids"],
    attention_mask=t["attention_mask"],
    token_type_ids=t["token_type_ids"])
  bert_out=bert(**t)

  predictions = bert_out["prediction_logits"]
  top10=torch.argsort(predictions,dim=2,descending=True)[:,:,:5] #why dim=2? what does [:,:,:20] do?
  #print(texts[0])
  #print(top5[0,4])

  #rev=reviews_masked.iloc[i,0]
  guesses=tokenizer.decode(top10[0, reviews_ans.iloc[i,0]])
  actual_ans= reviews_ans.iloc[i,1]
  print("Guesses:",guesses)
  print("And ans was ", actual_ans)
  if str(actual_ans) in guesses:
    print("Hooray! its in the top 10 guesses!!!")
    correct=correct+1
  else:
    print("oops! ITS NOT IN TOP 10 guesses!!! :()")

  print("------------------------------------------")
#an easy way to say the above would be bert(**t)

masked_position     int64
masked_words       object
dtype: object
masked_sentences    object
dtype: object
Guesses: see watch have get receive
And ans was  write
oops! ITS NOT IN TOP 10 guesses!!! :()
------------------------------------------
Guesses: ? Start Next Go Now
And ans was  begin.
oops! ITS NOT IN TOP 10 guesses!!! :()
------------------------------------------
Guesses: good great fine excellent strong
And ans was  and
oops! ITS NOT IN TOP 10 guesses!!! :()
------------------------------------------
Guesses: very some the pretty really
And ans was  the
Hooray! its in the top 10 guesses!!!
------------------------------------------
Guesses: really just certainly sure probably
And ans was  really
Hooray! its in the top 10 guesses!!!
------------------------------------------
Guesses: classic many enough more "
And ans was  live-action,
oops! ITS NOT IN TOP 10 guesses!!! :()
------------------------------------------
Guesses: expecting expected expect hoping getting
And ans was

In [ ]:
accuracy=correct/total*100
print("The accuracy of our model when it tries to guess our word being in the top 10: ", accuracy)

The accuracy of our model when it tries to guess our word being in the top 10:  30.0
